In [3]:
# Step 1: Import Required Libraries
import heapq
import random

# Step 2: Define the Graph with City Distances
# Example: Cities in California and their approximate distances (in miles)
city_graph = {
    'Los Angeles': {'San Francisco': 382, 'San Diego': 120, 'Sacramento': 384, 'San Jose': 340},
    'San Francisco': {'Los Angeles': 382, 'San Diego': 500, 'Sacramento': 88, 'San Jose': 47},
    'San Diego': {'Los Angeles': 120, 'San Francisco': 500, 'Sacramento': 520, 'San Jose': 480},
    'Sacramento': {'Los Angeles': 384, 'San Francisco': 88, 'San Diego': 520, 'San Jose': 115},
    'San Jose': {'Los Angeles': 340, 'San Francisco': 47, 'San Diego': 480, 'Sacramento': 115}
}

# List of cities
cities = list(city_graph.keys())

# Heuristic function (straight-line distance or any other estimate)
def heuristic(city, goal):
    # Example heuristic: straight-line distance (not accurate, but for demonstration)
    heuristic_distances = {
        'Los Angeles': {'San Francisco': 382, 'San Diego': 120, 'Sacramento': 384, 'San Jose': 340},
        'San Francisco': {'Los Angeles': 382, 'San Diego': 500, 'Sacramento': 88, 'San Jose': 47},
        'San Diego': {'Los Angeles': 120, 'San Francisco': 500, 'Sacramento': 520, 'San Jose': 480},
        'Sacramento': {'Los Angeles': 384, 'San Francisco': 88, 'San Diego': 520, 'San Jose': 115},
        'San Jose': {'Los Angeles': 340, 'San Francisco': 47, 'San Diego': 480, 'Sacramento': 115}
    }
    # Return 0 if the city or goal is not in the heuristic_distances dictionary
    return heuristic_distances.get(city, {}).get(goal, 0)

# Step 3: Implement A* Search Algorithm
def a_star_search(graph, start, end):
    """
    Finds the shortest path between two cities using A* search algorithm.
    :param graph: The graph representing the cities and their distances.
    :param start: The starting city.
    :param end: The destination city.
    :return: The shortest path and its total distance.
    """
    # Priority queue to store (f_score, g_score, city)
    pq = [(0 + heuristic(start, end), 0, start)]
    # Dictionary to store the shortest distance to each city
    g_scores = {city: float('inf') for city in graph}
    g_scores[start] = 0
    # Dictionary to store the path
    previous_nodes = {city: None for city in graph}
    
    while pq:
        _, current_g_score, current_city = heapq.heappop(pq)
        
        # If we reach the destination city, return the path
        if current_city == end:
            path = []
            while current_city is not None:
                path.append(current_city)
                current_city = previous_nodes[current_city]
            return path[::-1], g_scores[end]
        
        # If the current g_score is greater than the recorded g_score, skip
        if current_g_score > g_scores[current_city]:
            continue
        
        # Explore neighbors
        for neighbor, distance in graph[current_city].items():
            tentative_g_score = current_g_score + distance
            if tentative_g_score < g_scores[neighbor]:
                g_scores[neighbor] = tentative_g_score
                f_score = tentative_g_score + heuristic(neighbor, end)
                previous_nodes[neighbor] = current_city
                heapq.heappush(pq, (f_score, tentative_g_score, neighbor))
    
    return None, float('inf')

# Step 4: Define Helper Functions for Genetic Algorithm
def calculate_total_distance(route):
    """
    Calculates the total distance of a route using A* search algorithm.
    :param route: A list of cities in the order of visitation.
    :return: The total distance of the route.
    """
    total_distance = 0
    for i in range(len(route) - 1):
        start = route[i]
        end = route[i + 1]
        _, distance = a_star_search(city_graph, start, end)
        total_distance += distance
    return total_distance

def generate_random_route(cities):
    """
    Generates a random route (permutation of cities).
    :param cities: A list of cities.
    :return: A random route.
    """
    route = cities.copy()
    random.shuffle(route)
    return route

# Step 5: Implement the Genetic Algorithm
def genetic_algorithm(cities, population_size=50, generations=100, mutation_rate=0.01):
    """
    Optimizes the delivery route using a genetic algorithm.
    :param cities: A list of cities to visit.
    :param population_size: The number of routes in each generation.
    :param generations: The number of generations to evolve.
    :param mutation_rate: The probability of mutation.
    :return: The best route and its total distance.
    """
    # Step 1: Initialize the population
    population = [generate_random_route(cities) for _ in range(population_size)]
    
    for generation in range(generations):
        # Step 2: Evaluate the fitness of each route
        fitness_scores = [1 / calculate_total_distance(route) for route in population]
        
        # Step 3: Select the best routes (parents)
        parents = []
        for _ in range(population_size):
            # Select two routes based on fitness scores
            parent1, parent2 = random.choices(population, weights=fitness_scores, k=2)
            parents.append((parent1, parent2))
        
        # Step 4: Create the next generation (crossover)
        next_generation = []
        for parent1, parent2 in parents:
            # Perform crossover (e.g., order crossover)
            crossover_point = random.randint(1, len(cities) - 1)
            child = parent1[:crossover_point] + [city for city in parent2 if city not in parent1[:crossover_point]]
            next_generation.append(child)
        
        # Step 5: Mutate some routes
        for i in range(len(next_generation)):
            if random.random() < mutation_rate:
                # Swap two random cities in the route
                idx1, idx2 = random.sample(range(len(cities)), 2)
                next_generation[i][idx1], next_generation[i][idx2] = next_generation[i][idx2], next_generation[i][idx1]
        
        # Replace the old population with the new generation
        population = next_generation
    
    # Step 6: Find the best route in the final population
    best_route = min(population, key=calculate_total_distance)
    best_distance = calculate_total_distance(best_route)
    
    return best_route, best_distance

# Step 6: Test the Hybrid Solution
if __name__ == "__main__":
    # Display available cities
    print("Available Cities:")
    print(", ".join(cities))
    
    # Let the user enter the starting and ending points
    start_city = input("Enter the starting city: ").strip()
    end_city = input("Enter the ending city: ").strip()
    
    # Validate user input
    if start_city not in cities or end_city not in cities:
        print("Invalid city names. Please choose from the available cities.")
    else:
        # Define the list of cities to visit (including start and end points)
        delivery_stops = [start_city, end_city]
        
        # Find the optimal route using the genetic algorithm
        optimal_route, total_distance = genetic_algorithm(delivery_stops)
        
        # Print the results
        print("\nOptimal Delivery Route (A* Search + Genetic Algorithm):")
        print(" -> ".join(optimal_route))
        print(f"Total Distance: {total_distance} miles")

Available Cities:
Los Angeles, San Francisco, San Diego, Sacramento, San Jose


Enter the starting city:  Los Angeles
Enter the ending city:  Sacramento



Optimal Delivery Route (A* Search + Genetic Algorithm):
Los Angeles -> Sacramento
Total Distance: 384 miles
